In [ ]:
import os
import sqlite3
import pandas as pd
import numpy as np
import sys
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from nltk import bigrams
from nltk.probability import FreqDist

import seaborn as sns
from datetime import datetime
import requests
import re
import csv
import sweetviz as sv
from tqdm.auto import tqdm

from concurrent.futures import ThreadPoolExecutor
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import LongformerConfig, LongformerModel
import tensorflow as tf
import itertools
from collections import Counter

os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Extract

## 1. Datensammlung Webseite "Letzte Generation"

In [ ]:
STORAGE_PATH = os.path.join("output", "data-lake")
DB_PATH = os.path.join(STORAGE_PATH, "artikel.db")

In [ ]:
# Scrapen aller Artikel auf 8 Seiten und Speichern in Sqlite-Datenbank
# Funktion zum Scrapen der Standard-Seiten 1-7
def scrape_standard_page(soup, c, current_date):
    article_count = 0
    for element in soup.find_all(["strong", "span"]):
        if element.name == "strong" and element.get("id", "").startswith("date_"):
            current_date = element.text.strip()
        elif element.name == "span" and "notranslate" in element.get("class", []):
            source_name = (
                element.text.split(":")[0].strip()
                if ":" in element.text
                else element.text.strip()
            )
            link_tag = element.find_next("a")
            article_url = link_tag["href"]
            article_title = link_tag.text.strip()

            # Überspringe JavaScript-Links
            if "javascript:void(0);" in article_url:
                continue

            # Zähler erhöhen und Daten in die Datenbank einfügen
            article_count += 1
            c.execute(
                "INSERT INTO Artikel VALUES (?, ?, ?, ?)",
                (current_date, source_name, article_title, article_url),
            )
    return article_count


# Funktion zum Scrapen der speziellen Seite (Seite 8)
def scrape_special_page(soup, c):
    article_count = 0
    current_date = None
    for element in soup.find_all(["strong", "b", "a"]):
        if element.name in ["strong", "b"]:
            date_match = re.match(r"\d{1,2}\.\d{1,2}\.\d{4}", element.text.strip())
            if date_match:
                current_date = date_match.group()

        elif element.name == "a" and element.has_attr("href") and current_date:
            article_url = element["href"]
            article_title = element.text.strip()
            preceding_text = element.find_previous_sibling(string=True)
            source_name = (
                preceding_text.split(":")[0].strip()
                if preceding_text and ":" in preceding_text
                else "Unbekannt"
            )

            # Überspringe JavaScript-Links
            if "javascript:void(0);" in article_url:
                continue

            article_count += 1
            c.execute(
                "INSERT INTO Artikel VALUES (?, ?, ?, ?)",
                (current_date, source_name, article_title, article_url),
            )
    return article_count


# Hauptteil des Skripts
conn = sqlite3.connect(os.path.join(STORAGE_PATH, "artikel.db"))
c = conn.cursor()
c.execute(
    """CREATE TABLE IF NOT EXISTS Artikel
             (date text, source_name text, article_title text, article_url text)"""
)

# Ermittle die Anzahl der Seiten
response = requests.get("https://letztegeneration.org/presse/berichterstattung-")
soup = BeautifulSoup(response.text, "html.parser")
pagination_links = soup.find_all(
    "a", href=lambda href: href and "berichterstattung-" in href
)
highest_page_number = 0
for link in pagination_links:
    try:
        page_number = int(link["href"].split("-")[-1])
        highest_page_number = max(highest_page_number, page_number)
    except ValueError:
        continue

# Scrapen aller Seiten
for i in range(1, highest_page_number + 1):
    url = f"https://letztegeneration.org/presse/berichterstattung-{i}/"
    response = requests.get(url)
    page_soup = BeautifulSoup(response.text, "html.parser")
    current_date = None

    # Überprüfe, welche Scraping-Methode anzuwenden ist
    if page_soup.find("div", class_="elementor-element"):
        article_count = scrape_special_page(page_soup, c)
    else:
        article_count = scrape_standard_page(page_soup, c, current_date)

    print(f"URL: {url} - Anzahl der Treffer: {article_count}")

conn.commit()
conn.close()

In [ ]:
# Aufruf aller html-URL aus Datenbank und paralleles Scrapen der html-Dateien und Speichern der Resultate in Datenbank

# Verbindung zur Datenbank herstellen
conn = sqlite3.connect(DB_PATH)
c = conn.cursor()

# Überprüfen, ob die Spalte 'html_content' bereits existiert, und falls nicht, hinzufügen
try:
    c.execute("SELECT html_content FROM Artikel LIMIT 1")
except sqlite3.OperationalError:
    c.execute("ALTER TABLE Artikel ADD COLUMN html_content TEXT")

conn.commit()


def fetch_html(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return url, response.text
        else:
            return url, None
    except Exception as e:
        return url, None


def write_error_to_csv(url):
    with open(os.path.join("output", "errors.csv"), "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([url])


def update_db(url, html_content):
    try:
        c.execute(
            "UPDATE Artikel SET html_content = ? WHERE article_url = ?",
            (html_content, url),
        )
        conn.commit()
    except Exception as e:
        write_error_to_csv(url)


# URLs aus der Datenbank abrufen
c.execute("SELECT article_url FROM Artikel")
urls = c.fetchall()

#Für Testzwecke: Liste auf die ersten 100 URLs begrenzen
#urls = urls[:1000]

# Paralleles Ausführen der Scraping-Funktion
with ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(fetch_html, [url[0] for url in urls])

    completed = 0
    for url, html_content in results:
        if html_content:
            update_db(url, html_content)
        else:
            write_error_to_csv(url)

        completed += 1
        if completed % 100 == 0:
            print(f"Bisher verarbeitete URLs: {completed}")

conn.close()

## Transform

### Data Cleansing

In [ ]:
# Entfernen von HTML_Tags aus html_content und Anlegen neuer .csv mit | als delimiter, um es in Tableau Prep einlesen zu können
# Funktion, um HTML-Tags zu entfernen
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# Verbindung zur SQLite-Datenbank herstellen
conn = sqlite3.connect(DB_PATH)

# Laden der Daten aus der Datenbank
df = pd.read_sql_query("SELECT * FROM Artikel", conn)

# Bereinigen der Daten (Beispiel: Entfernen von Nullwerten)
df = df.dropna()

# HTML-Tags aus der gewünschten Spalte entfernen
# Anwendung von tqdm für den Fortschrittsbalken
df['html_content'] = [remove_html_tags(x) if x else x for x in tqdm(df['html_content'], desc="HTML-Tags entfernen")]

# Speichern in einer neuen CSV-Datei mit alternativem Delimiter
df.to_csv(os.path.join("output", 'bereinigte_test.csv'), sep='|', index=False, encoding='utf-8')

conn.close()

### Die Datei 'bereinigte_test.csv' wurde in Tableau Prep geladen, die Daten bereinigt und normalisiert.

## Load

### Die bereinigten Daten wurden als '240102_artikel_output_cleansed.db' in eine Sqlite-Datenbank geladen. Mit dieser Datenbasis wurde weitergearbeitet ("data/ 240102_artikel_output_cleansed.db").

In [ ]:
SOURCES_PATH_FINAL = os.path.join("data", "240102_artikel_output_cleansed.db")
STORAGE_PATH_FINAL = os.path.join("output")

### Profiling report via Sweetviz 

In [ ]:
#profiling report
db_connection = sqlite3.connect(SOURCES_PATH_FINAL)

df = pd.read_sql_query("SELECT * FROM artikel_output_cleansed", db_connection)

report = sv.analyze(df)
report.show_html(os.path.join(STORAGE_PATH_FINAL, 'profiling_report_letztegeneration.html'))

db_connection.close()

## 3.1 Analyse: Verteilung der Häufigkeit nach Medium

In [ ]:
#Datenbankabfrage nach Verteilung der Häufigkeit nach Medium/source_name + Visualisierung
db_connection = sqlite3.connect(SOURCES_PATH_FINAL)
cursor = db_connection.cursor()

# SQL-Abfrage, um die Top 15 Medien zu erhalten
sql_top_media = """
    SELECT source_name, COUNT(*) AS anzahl_der_artikel
    FROM artikel_output_cleansed
    GROUP BY source_name
    ORDER BY anzahl_der_artikel DESC
    LIMIT 10;
"""
cursor.execute(sql_top_media)

# Ergebnisse abrufen
results = cursor.fetchall()

# Daten in DataFrame konvertieren
df = pd.DataFrame(results, columns=['Medium', 'Anzahl_der_Artikel'])

# Balkendiagramm mit Seaborn erstellen
plt.figure(figsize=(10, 6))
ax = sns.barplot(x="Medium", y="Anzahl_der_Artikel", data=df)
plt.xticks(rotation=45, fontsize=8) 
ax.set_xlabel("Medium", labelpad=15)
ax.set_ylabel("Anzahl der Artikel", labelpad=15)
ax.set_title("TOP 10 Medien mit der höchsten Artikelanzahl", fontsize=13)

# Werte an der Spitze der Balken anzeigen
for p in ax.patches:
    ax.annotate(f"{int(p.get_height())}", (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=10, xytext=(0, 5), textcoords='offset points')

# Oberen und rechten Achsenlinien ausblenden
sns.despine()

plt.tight_layout()
plt.show()
print(results)

db_connection.close()


## 3.1 Analyse: Verteilung der Häufigkeit nach Datum

In [ ]:
#Datenbankabfrage nach Verteilung der Häufigkeit nach Datum + Visualisierung
db_connection = sqlite3.connect(SOURCES_PATH_FINAL)
cursor = db_connection.cursor()

cursor.execute("""
    SELECT date_formatted, COUNT(*) AS anzahl_der_meldungen
    FROM sentiment
    GROUP BY date_formatted
    ORDER BY anzahl_der_meldungen DESC
    LIMIT 10;
""")

# Ergebnisse in ein Pandas DataFrame laden
df = pd.DataFrame(cursor.fetchall(), columns=["Datum", "Anzahl_der_Meldungen"])

# Diagramm erstellen, wenn Daten vorhanden sind
if not df.empty:
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x="Datum", y="Anzahl_der_Meldungen", data=df)
    ax.set_xlabel("Datum", labelpad=10)
    ax.set_ylabel("Anzahl der Meldungen", labelpad=10)
    ax.set_title("TOP 10 Tage mit der höchsten Artikelanzahl", fontsize=13)
    plt.xticks(rotation=45)
    
    # Werte an der Spitze der Balken anzeigen
    for p in ax.patches:
        ax.annotate(f"{int(p.get_height())}", (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', fontsize=10, xytext=(0, 5), textcoords='offset points')

    # Oberen und rechten Achsenlinien ausblenden
    sns.despine()

    plt.tight_layout()
    plt.show()
else:
    print("Keine Ergebnisse gefunden.")

print(df)
db_connection.close()


## 3.2 Analyse: Sentiment-Analyse

In [ ]:
# Heuristische Textbereinigungsfunktion und Sentiment-Analyse
def clean_text_heuristically(text):
    # Entferne Zeilenumbrüche
    text = text.replace('\n', ' ')

    # Abschnitte mit hoher Linkdichte entfernen
    text = re.sub(r'\bhttps?:\/\/\S+\b', '', text)  # Entfernt URLs

    # Schlüsselwörter entfernen, die auf irrelevante Abschnitte hinweisen
    keywords_to_remove = ['Kontakt', 'Werbung', 'Impressum', 'Datenschutz']
    for keyword in keywords_to_remove:
        text = re.sub(r'\b' + keyword + r'\b', '', text)

    return text

conn = sqlite3.connect(SOURCES_PATH_FINAL)
cursor = conn.cursor()

# Neue Tabelle "sentiment" anlegen
cursor.execute("""
    CREATE TABLE IF NOT EXISTS sentiment (
        date TEXT,
        date_formatted TEXT,
        source_name TEXT,
        article_title TEXT,
        article_url TEXT,
        html_content TEXT,
        sentiment_label TEXT,
        sentiment_score REAL
    )
""")

# Datensätze mit minds. 20 Zeichen Länge abrufen
cursor.execute("""
    SELECT date, date_formatted, source_name, article_title, article_url, html_content 
    FROM artikel_output_cleansed 
    WHERE LENGTH(html_content) > 20
""")
rows = cursor.fetchall()

# Sentiment-Modell initialisieren
pipe = pipeline("text-classification", model="mdraw/german-news-sentiment-bert")

# Sentiment-Analyse für jeden Datensatz durchführen und Ergebnisse in der Datenbank speichern, max. Tokenlänge 512
for row in tqdm(rows, desc="Analyse der Sentiments", unit="text"):
    date, date_formatted, source_name, article_title, article_url, html_content = row
    cleaned_text = clean_text_heuristically(html_content)  # Heuristische Textbereinigung
    truncated_text = cleaned_text[:512] if len(cleaned_text) > 512 else cleaned_text
    result = pipe(truncated_text)
    sentiment_label = result[0]['label']
    sentiment_score = result[0]['score']

    # Einfügen des Datensatzes in die Tabelle "sentiment"
    cursor.execute("""
        INSERT INTO sentiment (date, date_formatted, source_name, article_title, article_url, html_content, sentiment_label, sentiment_score) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (date, date_formatted, source_name, article_title, article_url, cleaned_text, sentiment_label, sentiment_score))

conn.commit()
conn.close()


In [ ]:
# Visualisierung der Ergebnisse der Sentiment-Analyse

conn = sqlite3.connect(SOURCES_PATH_FINAL)
query = "SELECT sentiment_score FROM sentiment"
df_output = pd.read_sql_query(query, conn)
conn.close()

# Zusammenfassende Statistiken generieren
print(df_output[["sentiment_score"]].describe())

# Visualisierung der Sentiment Scores
sns.histplot(df_output['sentiment_score'], kde=True)
plt.title('Verteilung der Sentiment Scores')
plt.xlabel('Sentiment Score')
plt.ylabel('Häufigkeit')
plt.show()

In [ ]:
# Berechnung der Anzahl der sentiment label
conn = sqlite3.connect(SOURCES_PATH_FINAL)
cursor = conn.cursor()

# Anfrage, um die Anzahl der verschiedenen Labels zu ermitteln
query = """
    SELECT sentiment_label, COUNT(*) 
    FROM sentiment 
    GROUP BY sentiment_label
"""
cursor.execute(query)
label_counts = cursor.fetchall()

# Ausgabe der Anzahl für jedes Label
for label, count in label_counts:
    print(f"Label '{label}': {count} mal vorhanden")

conn.close()


In [ ]:
# Sentiment-Analyse der TOP 10 Tage nach Artikelanzahl

# Heuristische Textbereinigungsfunktion
def clean_text_heuristically(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'\bhttps?:\/\/\S+\b', '', text)
    keywords_to_remove = ['Kontakt', 'Werbung', 'Impressum', 'Datenschutz']
    for keyword in keywords_to_remove:
        text = re.sub(r'\b' + keyword + r'\b', '', text)
    return text

# Datenbankverbindung herstellen
conn = sqlite3.connect(SOURCES_PATH_FINAL)
cursor = conn.cursor()

# Sentiment-Modell initialisieren
pipe = pipeline("text-classification", model="mdraw/german-news-sentiment-bert")

# Datumsliste für die TOP 10 Tage
top_10_dates = ['2023-05-25', '2023-07-13', '2022-11-24', '2022-11-09', '2022-11-04', 
                '2022-12-09', '2023-04-24', '2022-11-25', '2023-02-06', '2022-11-01']

# Durchschnittliche Sentiments für die TOP 10 Tage berechnen
for date in top_10_dates:
    cursor.execute("""
        SELECT html_content 
        FROM artikel_output_cleansed 
        WHERE date_formatted = ? AND LENGTH(html_content) > 20
    """, (date,))
    articles = cursor.fetchall()

    sentiments = []
    for article in articles:
        html_content = article[0]
        cleaned_text = clean_text_heuristically(html_content)
        truncated_text = cleaned_text[:512] if len(cleaned_text) > 512 else cleaned_text
        sentiment_result = pipe(truncated_text)
        sentiment_label = sentiment_result[0]['label']
        sentiments.append(sentiment_label)

    sentiment_counts = Counter(sentiments)
    if sentiment_counts:
        dominant_sentiment = sentiment_counts.most_common(1)[0]
        print(f"Datum: {date}, Dominantes Sentiment: {dominant_sentiment[0]} (Anzahl: {dominant_sentiment[1]})")
    else:
        print(f"Datum: {date}, Keine Artikel gefunden")

conn.close()


### Analyse der häufigsten Wörter in den Artikeln

In [ ]:
# Bereinigen von Stopwörtern und Zahlen
stopwords_url = "https://raw.githubusercontent.com/solariz/german_stopwords/master/german_stopwords_full.txt"
stopwords_list = requests.get(stopwords_url).text.split("\n")[9:]
# Spezifische Wörter zur Stopwörterliste hinzufügen
additional_stopwords = ["the", "generation", "loading" , "externer inhalt", "twitter", "facebook", "instagram", "impressum", "kontakt", "kleinanzeige", "externen inhalt", "news"]
stopwords_list.extend(additional_stopwords)

def clean_text(text):
    """Bereinigt den Text, indem Stopwörter und kurze Wörter/Zahlen entfernt werden."""
    words = re.findall(r'\w+', text.lower())
    return ' '.join(word for word in words if word not in stopwords_list and len(word) > 2 and not word.isdigit())


In [ ]:
# Analyse der häufigsten Wörter
def count_most_common_words(db_path, table_name, column_name, num_words):
    """ Laden der Daten, Text bereinigen und zurückgeben der häufigsten Wörter bis zu einer festgelegten Anzahl"""
    db_connection = sqlite3.connect(db_path)

    query = f"SELECT {column_name} FROM {table_name} LIMIT 100"
    df = pd.read_sql_query(query, db_connection)

    db_connection.close()

    # Parallelisiere die Textbereinigung und Fortschrittsbalken
    with ThreadPoolExecutor() as executor:
        cleaned_texts = list(tqdm(executor.map(clean_text, df[column_name].dropna()), total=df[column_name].notna().sum()))

    # Kombiniere alle bereinigten Texte in einer Zeichenkette
    all_text = ' '.join(cleaned_texts)

    most_common_words = Counter(all_text.split()).most_common(num_words)

    return most_common_words

most_common_words = count_most_common_words(SOURCES_PATH_FINAL, "artikel_output_cleansed", "html_content", 20)

# Ausgabe der 20 häufigsten Worte
for word, count in most_common_words:
    print(f"{word}: {count}")


### Profiling report via Sweetviz nach Sentiment-Analyse

In [ ]:
# profiling report auf basis der Sentiment-Ergebnisse
db_connection = sqlite3.connect(SOURCES_PATH_FINAL)
df = pd.read_sql_query("SELECT * FROM sentiment", db_connection)

report = sv.analyze(df)

report.show_html('output/profiling_report_letztegeneration_2.html')

db_connection.close()

## 3.3 Analyse: Assoziationsanalyse

In [ ]:
# Häufige Bigramme für bestimmte Aktionsformen extrahieren

def get_frequent_bigrams_for_action(cursor, action, n_most_common=5):
    cursor.execute("SELECT html_content FROM sentiment WHERE html_content LIKE ?", ('%' + action + '%',))
    texts = [clean_text(BeautifulSoup(row[0], "html.parser").get_text()) for row in cursor.fetchall()]
    all_words = list(itertools.chain.from_iterable([text.split() for text in texts]))
    bigram_counts = Counter(bigrams(all_words))
    return bigram_counts.most_common(n_most_common)

def main():
    conn = sqlite3.connect(SOURCES_PATH_FINAL)
    cursor = conn.cursor()

    actions = [
        "Containern",
        "Straßenblockade",
        "Gebäude blockieren",
        "Kunstaktion",
        "Veranstaltung stören",
        "gegen Reiche"
    ]

    results = []
    for action in tqdm(actions, desc="Verarbeite Anfrage"):
        common_bigrams = get_frequent_bigrams_for_action(cursor, action)
        results.extend([(action, ' '.join(bigram), freq) for bigram, freq in common_bigrams])

    conn.close()

    # Ergebnisse in CSV-Datei speichern
    with open(os.path.join("output", 'associations_analysis_results.csv'), 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Aktion', 'Bigramm', 'Häufigkeit'])
        writer.writerows(results)

    #print(results)
    return results

# Die main-Funktion aufrufen und Ergebnisse in einer Variable speichern
association_results = main()
print(association_results)


In [ ]:
# Visualisierung der Ergebnisse der Assoziationsanalyse über ein Bubble Chart
# Erstellen eines DataFrame aus den Ergebnissen der Analyse
df = pd.DataFrame(association_results, columns=["Aktion", "Bigramm", "Häufigkeit"])

# Pivot-Tabelle für die Heatmap
pivot_table = df.pivot("Aktion", "Bigramm", "Häufigkeit")

# Bubble Chart erstellen
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df, x="Bigramm", y="Aktion", size="Häufigkeit", legend=False, sizes=(100, 2000))
plt.title("Bubble Chart der Bigramme und Aktionen", fontsize=13)
plt.xticks(rotation=45, fontsize=8)
sns.despine()
plt.show()
